In [250]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter

import sklearn
import sklearn.model_selection
import sklearn.ensemble
import sklearn.metrics
import sklearn.model_selection

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [251]:
train = pd.read_csv('Train.csv.zip', compression='zip')
test = pd.read_csv('Test.csv.zip', compression='zip')
# train.columns

In [252]:
# Save the 'Id' column
train_ID = train['id']
test_ID = test['id']

# Now drop the 'Id' column since it's unnecessary for the prediction process.
train.drop("id", axis = 1, inplace = True)
test.drop("id", axis = 1, inplace = True)

In [253]:
# Combining Datasets
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.price.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['price'], axis=1, inplace=True)
print("Train data size is : {}".format(train.shape))
print("Test data size is : {}".format(test.shape))
print("Combined dataset size is : {}".format(all_data.shape))

Train data size is : (100000, 24)
Test data size is : (100000, 23)
Combined dataset size is : (200000, 23)


/home/axreldable/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [254]:
# настройка параметров
all_data[['build_tech']] = all_data[['build_tech']].replace(np.nan, 0.5)
all_data[['metro_dist']] = all_data[['metro_dist']].replace(np.nan, 15.0)
all_data[['g_lift']] = all_data[['g_lift']].replace(np.nan, 0.5)

In [255]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(200000, 58)


In [256]:
# all_data.drop("date", axis = 1, inplace = True)
# all_data.drop("kw1", axis = 1, inplace = True)
# all_data.drop("kw2", axis = 1, inplace = True)
# all_data.drop("kw3", axis = 1, inplace = True)
# all_data.drop("kw4", axis = 1, inplace = True)
# all_data.drop("kw5", axis = 1, inplace = True)
# all_data.drop("kw6", axis = 1, inplace = True)
# all_data.drop("kw7", axis = 1, inplace = True)
# all_data.drop("kw8", axis = 1, inplace = True)
# all_data.drop("kw9", axis = 1, inplace = True)
# all_data.drop("kw10", axis = 1, inplace = True)
# all_data.drop("kw11", axis = 1, inplace = True)
# all_data.drop("kw12", axis = 1, inplace = True)
# all_data.drop("kw13", axis = 1, inplace = True)

In [257]:
# Check if there are any missing values left
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


In [258]:
all_data
all_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 58 columns):
area               200000 non-null int64
balcon             200000 non-null int64
build_tech         200000 non-null float64
floor              200000 non-null int64
g_lift             200000 non-null float64
kw1                200000 non-null int64
kw10               200000 non-null int64
kw11               200000 non-null int64
kw12               200000 non-null int64
kw13               200000 non-null int64
kw2                200000 non-null int64
kw3                200000 non-null int64
kw4                200000 non-null int64
kw5                200000 non-null int64
kw6                200000 non-null int64
kw7                200000 non-null int64
kw8                200000 non-null int64
kw9                200000 non-null int64
metro_dist         200000 non-null float64
n_photos           200000 non-null int64
rooms              200000 non-null int64
street_id          200

In [259]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [260]:
X, Xt, y, yt = sklearn.model_selection.train_test_split(train, y_train, test_size=0.3, random_state=0)

In [261]:
# Cross-validation with k-folds
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= -cross_val_score(model, train.values, y_train, scoring="neg_mean_absolute_error", cv = kf)
    return(rmse)

In [262]:
# lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1)) # 1692737.2141
# lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.001, random_state=1)) # 1692737.2140
# lasso = make_pipeline(RobustScaler(), Lasso(alpha = 0.1, random_state=1)) # 1692737.2017

# date include
# lasso = make_pipeline(RobustScaler(), Lasso(alpha=5, normalize=False, max_iter=1000, warm_start=False, random_state=1)) # 1692972.0131
# lasso = make_pipeline(RobustScaler(), Lasso(alpha=5, normalize=False, max_iter=500, warm_start=False, random_state=1)) # 1693997.8751
# lasso = make_pipeline(RobustScaler(), Lasso(alpha=5, normalize=False, max_iter=5000, warm_start=False, random_state=1)) # 1692268.2914

# all priznaki
lasso = make_pipeline(RobustScaler(), Lasso(alpha=5, normalize=False, max_iter=5000, warm_start=False, random_state=1)) # 1692268.2914

In [263]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 1691357.5555 (26581.8762)



In [267]:
clf = Lasso(alpha=5, normalize=False, max_iter=5000, warm_start=False, random_state=1)

In [268]:
clf.fit(X, y)

Lasso(alpha=5, copy_X=True, fit_intercept=True, max_iter=5000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False)

In [269]:
clf.coef_

array([ 2.01553279e+05,  4.29368121e+05,  5.22254990e+05,  5.76082417e+04,
       -3.05961099e+03,  2.31807418e+05,  6.02964060e+05,  1.55009807e+06,
       -4.31860431e+05,  2.72455334e+05, -9.24753915e+04, -2.72117244e+05,
       -2.74598654e+05,  5.13248739e+04,  3.53140353e+05,  8.41057171e+05,
       -1.09129457e+05,  3.67426076e+04, -5.34125021e+04,  2.24759818e+05,
       -1.80133222e+06, -8.48817287e+02, -1.04790936e+05, -1.68292509e+05,
       -9.66363763e+04, -7.62892559e+04, -5.77824004e+04,  0.00000000e+00,
       -3.94684284e+04, -8.98548228e+03,  7.14783786e+04,  9.50589189e+04,
        1.24988472e+04,  2.32689923e+05,  3.15936227e+04,  1.21546616e+05,
       -2.50443245e+04,  9.33709282e+04, -3.31117287e+04,  7.42986888e+04,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [264]:
# model_xgb.fit(X, y)

# pred = model_xgb.predict(Xt)

# score = sklearn.metrics.mean_absolute_error(yt, pred)

# print('XGBRegressor max_depth=13 n_estimators=10000: %s' % (score))